###### **Load the Data**

In [ ]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config() 
dataset = Dataset.get_by_name(ws, name="OpenI_dataset", version='1')

dataset.download(target_path="data/", overwrite=True)

In [2]:
import pandas as pd
import os

reports_df = pd.read_csv("data/indiana_reports.csv")
projections_df = pd.read_csv("data/indiana_projections.csv")

print(reports_df.head())
print(projections_df.head())


   uid                                               MeSH  \
0    1                                             normal   
1    2  Cardiomegaly/borderline;Pulmonary Artery/enlarged   
2    3                                             normal   
3    4  Pulmonary Disease, Chronic Obstructive;Bullous...   
4    5  Osteophyte/thoracic vertebrae/multiple/small;T...   

                                            Problems  \
0                                             normal   
1                      Cardiomegaly;Pulmonary Artery   
2                                             normal   
3  Pulmonary Disease, Chronic Obstructive;Bullous...   
4                         Osteophyte;Thickening;Lung   

                                               image  \
0                          Xray Chest PA and Lateral   
1                Chest, 2 views, frontal and lateral   
2                          Xray Chest PA and Lateral   
3  PA and lateral views of the chest XXXX, XXXX a...   
4               

In [3]:
reports_df = reports_df.dropna(subset=['findings', 'impression'])
projections_df['projection'] = projections_df['projection'].str.lower()
frontal_df = projections_df[projections_df['projection'] == 'frontal']

merged_df = pd.merge(reports_df, frontal_df, on='uid')
merged_df.head()

,uid,MeSH,Problems,image,indication,comparison,findings,impression,filename,projection
0,1,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.,1_IM-0001-4001.dcm.png,frontal
1,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.,2_IM-0652-1001.dcm.png,frontal
2,4,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...,4_IM-2050-1001.dcm.png,frontal
3,5,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.,5_IM-2117-1003002.dcm.png,frontal
4,6,normal,normal,"PA and Lateral Chest. XXXX, XXXX at XXXX",Evaluate for infection,"XXXX, XXXX",Heart size and mediastinal contour are within ...,No acute cardiopulmonary findings.,6_IM-2192-1001.dcm.png,frontal


In [4]:
merged_df["report_text"] = merged_df["findings"].fillna('') + " " + merged_df["impression"].fillna('')
merged_df = merged_df.drop_duplicates(subset=['filename'])
merged_df = merged_df.reset_index(drop=True)
merged_df.head()

,uid,MeSH,Problems,image,indication,comparison,findings,impression,filename,projection,report_text
0,1,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.,1_IM-0001-4001.dcm.png,frontal,The cardiac silhouette and mediastinum size ar...
1,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.,2_IM-0652-1001.dcm.png,frontal,Borderline cardiomegaly. Midline sternotomy XX...
2,4,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...,4_IM-2050-1001.dcm.png,frontal,There are diffuse bilateral interstitial and a...
3,5,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.,5_IM-2117-1003002.dcm.png,frontal,The cardiomediastinal silhouette and pulmonary...
4,6,normal,normal,"PA and Lateral Chest. XXXX, XXXX at XXXX",Evaluate for infection,"XXXX, XXXX",Heart size and mediastinal contour are within ...,No acute cardiopulmonary findings.,6_IM-2192-1001.dcm.png,frontal,Heart size and mediastinal contour are within ...


In [5]:
merged_df.to_csv("data/cleaned_metadata.csv", index=False)

merged_df[["uid", "filename", "report_text"]].head()

,uid,filename,report_text
0,1,1_IM-0001-4001.dcm.png,The cardiac silhouette and mediastinum size ar...
1,2,2_IM-0652-1001.dcm.png,Borderline cardiomegaly. Midline sternotomy XX...
2,4,4_IM-2050-1001.dcm.png,There are diffuse bilateral interstitial and a...
3,5,5_IM-2117-1003002.dcm.png,The cardiomediastinal silhouette and pulmonary...
4,6,6_IM-2192-1001.dcm.png,Heart size and mediastinal contour are within ...


In [6]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install quiet

ERROR: Could not find a version that satisfies the requirement quiet (from versions: none)
ERROR: No matching distribution found for quiet
Note: you may need to restart the kernel to use updated packages.


###### **Text Embeddings**

In [9]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

df = pd.read_csv("data/cleaned_metadata.csv")

model_name='dmis-lab/biobert-base-cased-v1.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [10]:
def get_biobert_embedding(text):
    inputs= tokenizer(text, return_tensors='pt', truncation = True,padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

###### **Generating Text Embeddings**

In [11]:
import numpy as np

embeddings = []

for text in tqdm(df["report_text"], desc="Generating BioBERT embeddings"):
    emb = get_biobert_embedding(text)
    embeddings.append(emb)

text_embeddings = np.stack(embeddings)

np.save("data/biobert_text_embeddings.npy", text_embeddings)

print("✅ Text embeddings generated and saved as 'biobert_text_embeddings.npy'")

Generating BioBERT embeddings:  99%|█████████▊| 3258/3301 [05:17<00:04,  9.69it/s]

✅ Text embeddings generated and saved as 'biobert_text_embeddings.npy'


##### **Image Embeddings with DenseNet**

In [12]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
from PIL import Image
from tqdm import tqdm
import torch
from torchvision import models, transforms

image_dir = "data/images/images_normalized/"

df = pd.read_csv("data/cleaned_metadata.csv")

df["image_path"] = df["filename"].apply(lambda x: os.path.join(image_dir, x))
df = df[df["image_path"].apply(os.path.exists)].reset_index(drop=True)

In [ ]:
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

densenet = models.densenet121(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(densenet.features.children()))
feature_extractor.eval()

In [ ]:
image_embeddings = []

for img_path in tqdm(df["image_path"], desc="Generating DenseNet embeddings"):
    try:
        image = Image.open(img_path).convert("RGB")
        input_tensor = image_transform(image).unsqueeze(0) 

        with torch.no_grad():
            features = feature_extractor(input_tensor)
            pooled = torch.nn.functional.adaptive_avg_pool2d(features, (1, 1))
            embedding = pooled.view(-1).numpy()
            image_embeddings.append(embedding)
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        image_embeddings.append(np.zeros(1024))  
print(image_embeddings)

In [18]:
import numpy as np

image_embeddings = np.stack(image_embeddings)
np.save("data/densenet_image_embeddings.npy", image_embeddings)

print("✅ Image embeddings saved as 'densenet_image_embeddings.npy'")

✅ Image embeddings saved as 'densenet_image_embeddings.npy'


###### **Load and Combine both the Embeddings**

In [19]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/cleaned_metadata.csv")


text_embeddings = np.load("data/biobert_text_embeddings.npy")
image_embeddings = np.load("data/densenet_image_embeddings.npy")

print("Text embeddings:", text_embeddings.shape)
print("Image embeddings:", image_embeddings.shape)
print("Metadata rows:", len(df))


Text embeddings: (3301, 768)
Image embeddings: (3301, 1024)
Metadata rows: 3301


In [20]:
!pip uninstall -y faiss-cpu faiss-gpu
!pip install faiss-cpu==1.7.4 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: faiss-cpu 1.7.4
Uninstalling faiss-cpu-1.7.4:
  Successfully uninstalled faiss-cpu-1.7.4
  Using cached faiss_cpu-1.7.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)


In [21]:
pip install numpy==1.23.5

Note: you may need to restart the kernel to use updated packages.


In [22]:
import faiss
import numpy as np

from sklearn.preprocessing import normalize


text_embeddings_norm = normalize(text_embeddings)
image_embeddings_norm = normalize(image_embeddings)


combined = np.concatenate([0.6 * text_embeddings_norm, 0.4 * image_embeddings_norm], axis=1)

combined_embeddings = np.array(combined, dtype=np.float32, copy=True)


In [23]:
import faiss
import numpy as np

combined_embeddings = np.ascontiguousarray(combined_embeddings, dtype=np.float32)

d = combined_embeddings.shape[1]
index = faiss.IndexFlatIP(d)
index.add(combined_embeddings)

print("✅ FAISS index successfully created with", index.ntotal, "entries")


✅ FAISS index successfully created with 3301 entries


In [24]:
def retrieve_similar_cases(query_text, image_embedding, top_k=5):
    from transformers import AutoTokenizer, AutoModel
    import torch

    tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
    model = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
    model.eval()

    inputs = tokenizer(query_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    query_text_emb = outputs.last_hidden_state[:, 0, :].squeeze().numpy()

    if image_embedding is not None:
        query_text_emb = normalize(query_text_emb.reshape(1, -1))
        image_embedding = normalize(image_embedding.reshape(1, -1))
        query_vector = np.concatenate([0.6 * text_embeddings_norm, 0.4 * image_embeddings_norm], axis=1)
    else:
        query_vector = normalize(query_text_emb.reshape(1, -1))

    D, I = index.search(query_vector.astype(np.float32), top_k)

    results = df.iloc[I[0]][["filename", "report_text"]]
    return results


In [25]:
results = retrieve_similar_cases(
    query_text="patient with diffuse bilateral opacities in the lungs",
    image_embedding=image_embeddings[0] 
)
print("Top matching cases:")
print(results)


Top matching cases:
                            filename  \
0             1_IM-0001-4001.dcm.png   
1744  2097_IM-0727-1001-0001.dcm.png   
2854       3466_IM-1683-1001.dcm.png   
2283       2784_IM-1220-1001.dcm.png   
2577       3137_IM-1476-1001.dcm.png   

                                            report_text  
0     The cardiac silhouette and mediastinum size ar...  
1744  The trachea is midline. Cardiomediastinal silh...  
2854  The cardiomediastinal silhouette is normal in ...  
2283  The heart is normal in size and contour. There...  
2577  The trachea is midline. Cardiomediastinal silh...  


###### **Format the RAG Context Prompt**

In [26]:
def build_rag_prompt(user_query,image_embedding, retrieved_reports):
    context = "\n\n".join(retrieved_reports["report_text"].tolist())
    prompt = f"""
You are a clinical assistant. Based on the following historical radiology reports and the user question, generate a clinical summary with a focus on possible diagnoses, severity, and recommended actions.

User Question:
{user_query}
{image_embedding}

Relevant Historical Reports:
{context}

Clinical Summary:
"""
    return prompt


###### **Using Hugging Face Transformers**

In [27]:
pip install sacremoses

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [28]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "microsoft/BioGPT-Large"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()

bio_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [29]:
def generate_response_biogpt(prompt):
    output = bio_pipeline(prompt, max_new_tokens=300, do_sample=True)[0]["generated_text"]
    return output

In [30]:
print("Files in current directory:", os.listdir("."))

Files in current directory: ['.amlignore', '.amlignore.amltmp', '.git', '.gradio', '.ipynb_aml_checkpoints', 'data', 'key', 'key.pub', 'Multimodal_AI.ipynb', 'multimodal_ai.ipynb.amltmp', 'requirements.txt', 'requirements.txt.amltmp', 'test_image.png', 'X-RayGPT-A-Biomedical-RAG-Chatbot-for-Radiology-Report-Generation-and-Severity-Prediction']


In [31]:
from PIL import Image
import torch
from torchvision import transforms


image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


from torchvision import models
densenet = models.densenet121(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(densenet.features.children()))
feature_extractor.eval()


def extract_embedding_from_image(image):
    image = Image.open(image_path).convert("RGB")
    input_tensor = image_transform(image).unsqueeze(0)

    with torch.no_grad():
        features = feature_extractor(input_tensor)
        pooled = torch.nn.functional.adaptive_avg_pool2d(features, (1, 1))
        embedding = pooled.view(-1).numpy()

    return embedding


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [32]:
image_path = "test_image.png" 
image_emb = extract_embedding_from_image(image_path)


In [33]:
query = "The patient exhibits signs of chronic obstructive pulmonary disease with increased interstitial markings."


top_k_reports = retrieve_similar_cases(query_text=query,image_embedding=image_emb, top_k=5)

prompt = build_rag_prompt(query, image_emb, top_k_reports)

response = generate_response_biogpt(prompt)  

print("🧠 AI Clinical Assistant Response:\n")
print(response)

🧠 AI Clinical Assistant Response:


You are a clinical assistant. Based on the following historical radiology reports and the user question, generate a clinical summary with a focus on possible diagnoses, severity, and recommended actions.

User Question:
The patient exhibits signs of chronic obstructive pulmonary disease with increased interstitial markings.
[-1.09165514e-04  2.69876327e-03  2.19170586e-03 ...  1.17157161e+00
  2.28229165e-01 -9.04489085e-02]

Relevant Historical Reports:
The cardiac silhouette and mediastinum size are within normal limits. There is no pulmonary edema. There is no focal consolidation. There are no XXXX of a pleural effusion. There is no evidence of pneumothorax. Normal chest x-XXXX.

The trachea is midline. Cardiomediastinal silhouette is normal. The lungs are clear without evidence of acute infiltrate or effusion. There is no pneumothorax. The visualized bony structures reveal no abnormalities. No acute cardiopulmonary abnormality seen on chest x-XXX

In [34]:
pip install gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install nbformat

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import gradio as gr

def multimodal_rag_chat(text_query, image):
    image_emb = extract_embedding_from_image(image) if image else None
    top_k = retrieve_similar_cases(text_query, image_embedding=image_emb)
    prompt = build_rag_prompt(text_query, top_k)
    response = generate_response_biogpt(prompt) 
    return response

iface = gr.Interface(
    fn=multimodal_rag_chat,
    inputs=[gr.Textbox(label="Clinical Query"), gr.Image(label="Optional Chest X-ray")],
    outputs="text",
    title="AI Clinical Assistant",
    description="Ask a question and optionally upload an X-ray image"
)

iface.launch(share=True)